# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [19]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [20]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tolanaro,-25.0319,46.9987,25.04,78,20,7.20,MG,1732676754
1,1,iqaluit,63.7506,-68.5145,-13.15,66,75,3.09,CA,1732676474
2,2,port alfred,-33.5906,26.8910,16.77,82,28,0.89,ZA,1732676757
3,3,georgetown,5.4112,100.3354,25.01,82,40,5.14,MY,1732676514
4,4,grytviken,-54.2811,-36.5092,0.90,75,65,5.35,GS,1732676759


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [21]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x='Lng',            
    y='Lat',              
    geo=True,            
    tiles='OSM',          
    size='Humidity',      
    color='Max Temp',     
    alpha=0.7,           
    frame_width=800,      
    frame_height=600,     
    title="City Weather Data"  
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Max Temp,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [23]:

# Narrow down cities that fit criteria and drop any results with null values
city_cleaned_df = city_data_df.dropna()

# Drop any rows with null values
ideal_weather_df = city_cleaned_df[(city_cleaned_df['Max Temp'] >= 15) & (city_cleaned_df['Max Temp'] <= 29.4) & (city_cleaned_df['Humidity'] < 40)]

# Display sample data
ideal_weather_df.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,ngaoundere,7.3167,13.5833,15.48,38,12,1.97,CM,1732676760
42,42,koundara,12.4833,-13.3000,20.91,39,0,0.72,GN,1732676804
54,54,jodhpur,26.2867,73.0300,22.16,34,0,1.85,IN,1732676563
56,56,sarakhs,36.5449,61.1577,15.40,36,0,3.15,IR,1732676820
65,65,sojat,25.9167,73.6667,21.44,33,2,1.78,IN,1732676830
86,86,koulikoro,14.0000,-7.7500,20.30,20,0,3.19,ML,1732676858
129,129,yanchep,-31.5500,115.6833,23.29,33,0,4.24,AU,1732676907
196,196,taoudenni,22.6783,-3.9836,19.89,31,32,4.63,ML,1732676983
209,209,wagar,16.1525,36.2032,20.63,23,1,4.18,SD,1732676998
241,241,nguigmi,14.2495,13.1092,18.53,36,4,5.87,NE,1732677035


### Step 3: Create a new DataFrame called `hotel_df`.

In [24]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = None

# Display sample data
hotel_df.sample(20)

,City,Country,Lat,Lng,Humidity,Hotel Name
393,villa regina,AR,-39.1000,-67.0667,20,None
372,menaka,ML,15.9182,2.4022,23,None
56,sarakhs,IR,36.5449,61.1577,36,None
507,bardai,NG,11.6500,7.2333,36,None
241,nguigmi,NE,14.2495,13.1092,36,None
552,alaghsas,NE,17.0187,8.0168,26,None
209,wagar,SD,16.1525,36.2032,23,None
86,koulikoro,ML,14.0000,-7.7500,20,None
196,taoudenni,ML,22.6783,-3.9836,31,None
456,timbuktu,ML,16.7735,-3.0074,17,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [32]:
# Set parameters to search for a hotel
category = "accomodation.hotel"
radius = 20000
limit = 1
params = {"categories": category, "limit": limit, "apiKey": geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row['Lng']
    latitude = row['Lat']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.sample(20)

Starting hotel search
ngaoundere - nearest hotel: No hotel found
koundara - nearest hotel: No hotel found
jodhpur - nearest hotel: No hotel found
sarakhs - nearest hotel: No hotel found
sojat - nearest hotel: No hotel found
koulikoro - nearest hotel: No hotel found
yanchep - nearest hotel: No hotel found
taoudenni - nearest hotel: No hotel found
wagar - nearest hotel: No hotel found
nguigmi - nearest hotel: No hotel found
raja - nearest hotel: No hotel found
menaka - nearest hotel: No hotel found
am timan - nearest hotel: No hotel found
villa regina - nearest hotel: No hotel found
reo - nearest hotel: No hotel found
timbuktu - nearest hotel: No hotel found
port augusta - nearest hotel: No hotel found
mehar - nearest hotel: No hotel found
bardai - nearest hotel: No hotel found
mamou - nearest hotel: No hotel found
alaghsas - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
334,raja,SS,8.4596,25.6780,39,No hotel found
209,wagar,SD,16.1525,36.2032,23,No hotel found
196,taoudenni,ML,22.6783,-3.9836,31,No hotel found
54,jodhpur,IN,26.2867,73.0300,34,No hotel found
42,koundara,GN,12.4833,-13.3000,39,No hotel found
515,mamou,GN,10.5000,-12.0000,33,No hotel found
241,nguigmi,NE,14.2495,13.1092,36,No hotel found
386,am timan,TD,11.0297,20.2827,31,No hotel found
129,yanchep,AU,-31.5500,115.6833,33,No hotel found
467,port augusta,AU,-32.5000,137.7667,32,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [29]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo=True, 
    tiles='OSM',
    frame_width = 1000, 
    frame_height = 600, 
    color = 'City', 
    size = 'Humidity',
    scale = 1.2,
    hover_cols=['City', 'Country', 'Hotel Name']
)

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)